In [1]:
# Import required libraries

import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from scipy.stats import randint as sp_randint
from keras.layers import Dropout
import random

Using TensorFlow backend.


In [2]:
# Import dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Make y_train and y_test categoricals
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [3]:
# Making a bigger corpus

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

In [6]:
# function to make a model

"""
def create_model(conv_layers=1,neurons=128):
    classifier = Sequential()
    classifier.add(Convolution2D(32, 3, 3, input_shape = x_train.shape[1:], activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    for i in range(0,conv_layers):
        classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
        classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Dropout(0.5))
    classifier.add(Flatten())
    classifier.add(Dense(output_dim = neurons, activation = 'relu'))
    classifier.add(Dense(output_dim = 10, activation = 'softmax'))
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return classifier
"""

def create_model(neurons=128,lstm_layers=1):
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
    model.add(Convolution2D(32, (3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    for i in range(0,lstm_layers):
        model.add(Convolution2D(64, (3, 3), padding='same'))
        model.add(Convolution2D(64, (3, 3)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(output_dim = neurons, activation = 'relu'))
    model.add(Dense(output_dim = 10, activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [7]:
"""
classifier=create_model(conv_layers=2,neurons=128)

# fits the model on batches with real-time data augmentation:
classifier.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=100)
"""

classifier=create_model()

# fits the model on batches with real-time data augmentation:
classifier.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=50)



/home/sebashc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/home/sebashc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/50
1563/1562 [==============================] - 82s 52ms/step - loss: 1.6717 - accuracy: 0.3972
Epoch 2/50
1563/1562 [==============================] - 81s 52ms/step - loss: 1.4205 - accuracy: 0.4943
Epoch 3/50
1563/1562 [==============================] - 81s 52ms/step - loss: 1.3341 - accuracy: 0.5272
Epoch 4/50
1563/1562 [==============================] - 81s 52ms/step - loss: 1.2882 - accuracy: 0.5443
Epoch 5/50
1563/1562 [==============================] - 82s 52ms/step - loss: 1.2556 - accuracy: 0.5566
Epoch 6/50
1563/1562 [==============================] - 82s 53ms/step - loss: 1.2310 - accuracy: 0.5649
Epoch 7/50
1563/1562 [==============================] - 82s 53ms/step - loss: 1.2176 - accuracy: 0.5739
Epoch 8/50
1563/1562 [==============================] - 83s 53ms/step - loss: 1.2002 - accuracy: 0.5769
Epoch 9/50
1563/1562 [==============================] - 82s 52ms/step - loss: 1.1907 - accuracy: 0.5818
Epoch 10/50
1563/1562 [==============================] - 83s 53m

In [10]:
# summary of the performance

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

Y_pred = classifier.predict(x_test)
Y_pred = np.argmax(Y_pred, axis = 1)

CM = confusion_matrix(np.argmax(y_test, axis = 1), Y_pred)
print("\n"+"Confusion Matrix:" + "\n")
print(CM)

print("\n"+"Classification Report"+"\n")
print(classification_report(Y_pred,np.argmax(y_test, axis = 1)))


Confusion Matrix:

[[524  84  25   2   3   1  23  57  50 231]
 [  3 729   0   1   0   0  15   2  20 230]
 [ 65  69 180   8  17  27 260 184  18 172]
 [ 33  92   8 102   7  63 283 120  25 267]
 [ 20  17  14   6 155  11 390 237   9 141]
 [ 10  55   5  34   8 236 199 172   9 272]
 [  7   9   2   5   3   3 881  15   3  72]
 [  8  15   4   2   7  13  60 760   4 127]
 [ 58  78   1   0   0   0   6   8 477 372]
 [  5  33   0   0   0   0   5   4   3 950]]

Classification Report

              precision    recall  f1-score   support

           0       0.52      0.71      0.60       733
           1       0.73      0.62      0.67      1181
           2       0.18      0.75      0.29       239
           3       0.10      0.64      0.18       160
           4       0.15      0.78      0.26       200
           5       0.24      0.67      0.35       354
           6       0.88      0.42      0.56      2122
           7       0.76      0.49      0.59      1559
           8       0.48      0.77     